In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pyperclip
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests as req
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
  # 웹드라이버 실행
driver = webdriver.Chrome()

# 제품 목록 페이지 로드
driver.get("https://www.flipkart.com/search?sid=j9e%2Fabm%2Fhzg&otracker=CLP_Filters&p%5B%5D=facets.brand%255B%255D%3DLG")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CGtC98')))

# URL을 저장할 리스트 초기화
url_list = []

# 요소 찾기
elements = driver.find_elements(By.CLASS_NAME, 'CGtC98')

# 각 요소에서 'href' 속성을 가져와 리스트에 추가 (sponsored 요소는 제외)
for element in elements:
    try:
        # 'f8qK5m' 클래스를 가진 자식 요소가 있는지 확인
        if element.find_elements(By.CLASS_NAME, 'f8qK5m'):
            # 'f8qK5m' 클래스를 가진 자식 요소가 있다면 skip
            continue
        # 자식 요소가 'f8qK5m' 클래스를 가지고 있지 않다면 href 추출
        url_value = element.get_attribute('href')
        if url_value:  # href 속성이 있는 경우만 추가
            url_list.append(url_value)
    except NoSuchElementException:
        # 자식 요소가 없으면 그냥 넘어가기
        continue 
        
# 리뷰 데이터를 저장할 리스트 초기화
all_reviews = []

for url in tqdm(url_list):
    try:
        driver.get(url)
        
        # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용...
        try:
            # XPath를 사용하여 제목 찾기. 제품 코드 별 XPATH 달라서 접근 불가... 제목에서 코드 추출하세용... 
            list_items = driver.find_elements(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span')
            
            # 텍스트 추출
            hp_data = [item.text for item in list_items]
            if not hp_data:
                hp_data = ["No data"]  # 데이터가 없을 경우 기본값
        except (NoSuchElementException, TimeoutException) as e:
            hp_data = ["Error fetching data"]  # 오류 발생 시 기본값
            
        # 리뷰 버튼 클릭 대기
        try:
            review_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_23J90q.RcXBOT")))
            if review_button.is_displayed():
                review_button.click()
        except (NoSuchElementException, TimeoutException):
            pass
        
        # 낮은 별점 접근
        try:
            low_rating_option = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#container > div > div.VLIitu > div > div > div.DOjaWF.gdgoEp.col-9-12 > div.DOjaWF.gdgoEp.col-12-12 > div > div:nth-child(2) > div > div > div > div > select > option:nth-child(4)")))
            low_rating_option.click()
        except (NoSuchElementException, TimeoutException) as e:
            print(f"Error selecting sort option for URL {url}: {e}")
        
        # 데이터 확보
        page_number = 1
        
        while True:
            try:
                # 페이지 URL 업데이트
                current_url = driver.current_url
                full_url = f'{current_url}&page={page_number}'
                driver.get(full_url)
                
                # ZmyHeo 클래스의 존재 여부를 확인
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZmyHeo')))
                
                # 데이터 추출
                text_elements = driver.find_elements(By.CLASS_NAME, 'ZmyHeo')
                state_elements = driver.find_elements(By.CLASS_NAME, 'MztJPv')
                date_elements = driver.find_elements(By.CLASS_NAME, '_2NsDsF')
                star_elements = driver.find_elements(By.XPATH, "//*[contains(@class, 'XQDdHH') and (contains(@class, 'Js30Fc') or contains(@class, 'Czs3gR') or contains(@class, 'Ga3i8K'))]")
                
                text_list = [elem.text for elem in text_elements]
                state_list = [elem.text for elem in state_elements]
                date_list = [elem.text for elem in date_elements]
                dates = date_list[1::2]
                star_list = [elem.text for elem in star_elements]
                
                # 리뷰 데이터를 모든 리뷰 리스트에 추가
                for t, s, d, r in zip(text_list, state_list, dates, star_list):
                    for data in hp_data:
                        all_reviews.append({
                            'text': t,
                            'state': s,
                            'date': d,
                            'star': r,
                            'hp_data': data  # 각각의 hp_data 항목을 추가
                        })

                # 페이지 번호 증가
                page_number += 1
                
            except (NoSuchElementException, TimeoutException) as e:
                print(f"Error on page {page_number} for URL {full_url}: {e}")
                break
    
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# 현재 창 닫기
driver.quit()

  8%|██████▍                                                                            | 1/13 [00:29<05:59, 29.95s/it]

Error processing URL https://www.flipkart.com/lg-185-l-direct-cool-single-door-5-star-refrigerator-base-drawer-smart-inverter-compressor-fast-ice-making/p/itmb63c41ba73d29?pid=RFRGHMVYEGSA9XTT&lid=LSTRFRGHMVYEGSA9XTTHMQO8T&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_4&otracker=CLP_Filters&iid=73fd4a34-c233-4ec0-a09a-57a293e55de0.RFRGHMVYEGSA9XTT.SEARCH&ssid=vvf2dx0s000000001724715606449: name 'NoSuchElementException' is not defined


 15%|████████████▊                                                                      | 2/13 [01:00<05:36, 30.57s/it]

Error processing URL https://www.flipkart.com/lg-185-l-direct-cool-single-door-4-star-refrigerator-base-drawer-smart-inverter-compressor/p/itm298afe5163996?pid=RFRGQTNNXNRTNJHV&lid=LSTRFRGQTNNXNRTNJHVKCZMYE&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_5&otracker=CLP_Filters&iid=73fd4a34-c233-4ec0-a09a-57a293e55de0.RFRGQTNNXNRTNJHV.SEARCH&ssid=vvf2dx0s000000001724715606449: name 'NoSuchElementException' is not defined


 23%|███████████████████▏                                                               | 3/13 [01:21<04:18, 25.81s/it]

...

Error processing URL https://www.flipkart.com/lg-398-l-frost-free-double-door-3-star-refrigerator/p/itm28069d1b23b41?pid=RFRH27M9E8SEEDXF&lid=LSTRFRH27M9E8SEEDXFHAQMRT&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_20&otracker=CLP_Filters&iid=73fd4a34-c233-4ec0-a09a-57a293e55de0.RFRH27M9E8SEEDXF.SEARCH&ssid=vvf2dx0s000000001724715606449: name 'NoSuchElementException' is not defined


 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [06:00<00:27, 27.15s/it]

Error processing URL https://www.flipkart.com/lg-185-l-direct-cool-single-door-2-star-refrigerator/p/itm48e75c89d4b57?pid=RFRGSBRVQEED8AUP&lid=LSTRFRGSBRVQEED8AUPJCXX0I&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_23&otracker=CLP_Filters&iid=73fd4a34-c233-4ec0-a09a-57a293e55de0.RFRGSBRVQEED8AUP.SEARCH&ssid=vvf2dx0s000000001724715606449: name 'NoSuchElementException' is not defined


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [06:32<00:00, 30.17s/it]

Error processing URL https://www.flipkart.com/lg-185-l-direct-cool-single-door-5-star-refrigerator-base-drawer-smart-inverter-compressor-connect-fast-ice-making/p/itm90b58e19bfe9f?pid=RFRGHMVY7PF7PWCM&lid=LSTRFRGHMVY7PF7PWCMXCJIUY&marketplace=FLIPKART&store=j9e%2Fabm%2Fhzg&srno=b_1_24&otracker=CLP_Filters&iid=73fd4a34-c233-4ec0-a09a-57a293e55de0.RFRGHMVY7PF7PWCM.SEARCH&ssid=vvf2dx0s000000001724715606449: name 'NoSuchElementException' is not defined


In [6]:
df_LG_refri = pd.DataFrame(all_reviews)
df_LG_refri

,text,state,date,star,hp_data
0,Very poor cooling,"Certified Buyer, Darbhanga",1 month ago,1,LG 185 L Direct Cool Single Door 5 Star Refrig...
1,Not good product,"Certified Buyer, Bangarapet",2 months ago,1,LG 185 L Direct Cool Single Door 5 Star Refrig...
2,Worst experience to buy filp kart... over heat...,"Certified Buyer, Coimbatore",5 months ago,1,LG 185 L Direct Cool Single Door 5 Star Refrig...
3,"Worst quality wast of money, no return option","Certified Buyer, Tirupati",6 months ago,1,LG 185 L Direct Cool Single Door 5 Star Refrig...
4,Refrigerator base not fitting properly,"Certified Buyer, Noida",6 months ago,1,LG 185 L Direct Cool Single Door 5 Star Refrig...
...,...,...,...,...,...
1142,Good product,"Certified Buyer, Ghaziabad","May, 2022",3,LG 185 L Direct Cool Single Door 5 Star Refrig...
1143,Ok,"Certified Buyer, Ariyalur District","Apr, 2022",3,LG 185 L Direct Cool Single Door 5 Star Refrig...
1144,Ok good,"Certified Buyer, Nagaon","Apr, 2022",3,LG 185 L Direct Cool Single Door 5 Star Refrig...
1145,Colour very bed,"Certified Buyer, Gurugram","Apr, 2022",3,LG 185 L Direct Cool Single Door 5 Star Refrig...


In [7]:
df_LG_refri.to_csv("df_LG_refri.csv")